# 3D Gaussian Splatting Results Viewer

This notebook allows you to visualize the results of your 3D Gaussian Splatting experiments. It provides:
1.  **Video Playback**: Watch the generated videos (Live, Splat, Flythrough).
2.  **Combined View**: Generate and view a side-by-side comparison of Input | Splat | Render.
3.  **Mesh Visualization**: View the extracted mesh flythrough.

## Instructions
1.  Run the setup cells.
2.  Select your experiment folder from the dropdown.
3.  Click "Generate Combined View" to create the mega-visualization.
4.  Use the tabs to switch between different views.

In [1]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Video, clear_output, HTML
from pathlib import Path
import glob

# Set output directory
OUTPUT_ROOT = Path("../output")

def get_experiments():
    """Find all experiment folders in output directory."""
    if not OUTPUT_ROOT.exists():
        return []
    return sorted([d.name for d in OUTPUT_ROOT.iterdir() if d.is_dir()])

print(f"Found experiments: {get_experiments()}")

Found experiments: ['environment_map']


In [2]:
def combine_videos(experiment_name):
    """
    Combines '1_live_rendering.mp4' and '2_splat_visualization.mp4' 
    into a single 'combined_view.mp4' with [Input | Splat | Render].
    """
    exp_dir = OUTPUT_ROOT / experiment_name
    live_path = exp_dir / "1_live_rendering.mp4"
    splat_path = exp_dir / "2_splat_visualization.mp4"
    output_path = exp_dir / "combined_view.mp4"
    
    if not live_path.exists() or not splat_path.exists():
        print(f"Error: Missing source videos in {exp_dir}")
        return None
        
    if output_path.exists():
        print(f"Combined video already exists: {output_path}")
        return output_path

    print("Generating combined video... This may take a minute.")
    
    # Open videos
    cap_live = cv2.VideoCapture(str(live_path))
    cap_splat = cv2.VideoCapture(str(splat_path))
    
    # Get properties
    width_live = int(cap_live.get(cv2.CAP_PROP_FRAME_WIDTH))
    height_live = int(cap_live.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap_live.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap_live.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Live video is [Input | Render] side-by-side
    # We assume Input is left half, Render is right half
    half_width = width_live // 2
    
    # Splat video is full frame
    width_splat = int(cap_splat.get(cv2.CAP_PROP_FRAME_WIDTH))
    height_splat = int(cap_splat.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Target dimensions
    # We want: [Input] [Splat] [Render]
    # Input: (half_width, height_live)
    # Splat: (width_splat, height_splat) -> Resize to match height_live if needed
    # Render: (half_width, height_live)
    
    # Resize splat to match live height if needed
    target_height = height_live
    scale_splat = target_height / height_splat
    target_width_splat = int(width_splat * scale_splat)
    
    total_width = half_width + target_width_splat + half_width
    
    # Output writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (total_width, target_height))
    
    frame_idx = 0
    while True:
        ret1, frame_live = cap_live.read()
        ret2, frame_splat = cap_splat.read()
        
        if not ret1 or not ret2:
            break
            
        # Split live frame
        frame_input = frame_live[:, :half_width]
        frame_render = frame_live[:, half_width:]
        
        # Resize splat frame
        if height_splat != target_height:
            frame_splat = cv2.resize(frame_splat, (target_width_splat, target_height))
            
        # Combine
        combined = np.hstack([frame_input, frame_splat, frame_render])
        
        # # Add labels
        # cv2.putText(combined, "INPUT", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        # cv2.putText(combined, "SPLAT VISUALIZATION", (half_width + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        # cv2.putText(combined, "RENDERED VIEW", (half_width + target_width_splat + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        out.write(combined)
        
        frame_idx += 1
        if frame_idx % 50 == 0:
            print(f"Processed {frame_idx}/{total_frames} frames...", end='\r')
            
    cap_live.release()
    cap_splat.release()
    out.release()
    print(f"\nDone! Saved to {output_path}")
    return output_path

In [ ]:
# UI Components
experiments = get_experiments()
dropdown = widgets.Dropdown(options=experiments, description='Experiment:')
btn_generate = widgets.Button(description='Generate Combined View', button_style='primary')
output_area = widgets.Output()

def show_video(path):
    if not os.path.exists(path):
        return HTML(f"<b>Video not found:</b> {path}")
    
    # Use relative path for HTML video tag
    rel_path = os.path.relpath(path, os.getcwd())
    return Video(rel_path, width=800, html_attributes="controls autoplay loop")

def on_click_generate(b):
    with output_area:
        clear_output()
        exp_name = dropdown.value
        if not exp_name:
            print("Please select an experiment.")
            return
            
        combine_videos(exp_name)
        display_results(exp_name)

def display_results(exp_name):
    with output_area:
        clear_output()
        exp_dir = OUTPUT_ROOT / exp_name
        
        # Define video paths
        videos = {
            "Combined View (Input | Splat | Render)": exp_dir / "combined_view.mp4",
            "Live Rendering": exp_dir / "1_live_rendering.mp4",
            "Splat Visualization": exp_dir / "2_splat_visualization.mp4",
            "Model Flythrough": exp_dir / "3_model_flythrough.mp4",
            "Mesh Flythrough": exp_dir / "4_mesh_flythrough.mp4"
        }
        
        # Create tabs
        children = []
        titles = []
        
        for title, path in videos.items():
            out = widgets.Output()
            with out:
                if path.exists():
                    display(show_video(str(path)))
                    print(f"Path: {path}")
                else:
                    print(f"Video not found: {path}")
                    if title == "Combined View (Input | Splat | Render)":
                        print("Click 'Generate Combined View' to create it.")
            children.append(out)
            titles.append(title)
            
        tabs = widgets.Tab()
        tabs.children = children
        for i, title in enumerate(titles):
            tabs.set_title(i, title)
            
        display(tabs)
        
        # Show metrics if available
        metrics_path = exp_dir / "metrics_summary.txt"
        if metrics_path.exists():
            print("\n" + "="*40)
            print("METRICS SUMMARY")
            print("="*40)
            with open(metrics_path, 'r') as f:
                print(f.read())

btn_generate.on_click(on_click_generate)

def on_change_dropdown(change):
    if change['type'] == 'change' and change['name'] == 'value':
        display_results(change['new'])

dropdown.observe(on_change_dropdown, names='value')

# Layout
display(widgets.VBox([
    widgets.HBox([dropdown, btn_generate]),
    output_area
]))

# Initial display
if experiments:
    display_results(experiments[0])

## Mesh Visualization (Interactive)

If you have `trimesh` and `pythreejs` installed, you can visualize the mesh interactively below. Otherwise, please refer to the "Mesh Flythrough" video above.

In [ ]:
try:
    import trimesh
    import pythreejs as p3
    
    def view_mesh_interactive(exp_name):
        mesh_path = OUTPUT_ROOT / exp_name / "final" / "mesh.ply"
        if not mesh_path.exists():
            print(f"Mesh not found: {mesh_path}")
            return
            
        print(f"Loading mesh from {mesh_path}...")
        mesh = trimesh.load(mesh_path)
        
        # Convert to pythreejs
        # This is a simplified viewer
        vertices = mesh.vertices
        faces = mesh.faces
        colors = mesh.visual.vertex_colors[:, :3] / 255.0
        
        geometry = p3.BufferGeometry(
            attributes={
                'position': p3.BufferAttribute(vertices.astype(np.float32), normalized=False),
                'index': p3.BufferAttribute(faces.astype(np.uint32).ravel(), normalized=False),
                'color': p3.BufferAttribute(colors.astype(np.float32), normalized=False),
            }
        )
        
        material = p3.MeshStandardMaterial(vertexColors='VertexColors', side='DoubleSide')
        mesh_obj = p3.Mesh(geometry=geometry, material=material)
        
        # Scene
        view_width = 800
        view_height = 600
        camera = p3.PerspectiveCamera(position=[0, 0, 5], aspect=view_width/view_height)
        key_light = p3.DirectionalLight(position=[0, 10, 10])
        ambient_light = p3.AmbientLight(intensity=0.5)
        
        scene = p3.Scene(children=[mesh_obj, camera, key_light, ambient_light])
        controller = p3.OrbitControls(controlling=camera)
        
        renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)
        display(renderer)

    # Add button for interactive mesh
    btn_mesh = widgets.Button(description='Load Interactive Mesh', button_style='info')
    mesh_out = widgets.Output()
    
    def on_click_mesh(b):
        with mesh_out:
            clear_output()
            view_mesh_interactive(dropdown.value)
            
    btn_mesh.on_click(on_click_mesh)
    display(widgets.VBox([btn_mesh, mesh_out]))
    
except ImportError:
    print("Interactive mesh visualization requires 'trimesh' and 'pythreejs'.")
    print("You can install them with: pip install trimesh pythreejs")
    print("For now, please use the 'Mesh Flythrough' video tab above.")

: 